for 8queens problem

In [1]:
import random

def random_queen_state():
    
    return [random.randint(0, 7) for _ in range(8)]

def calculate_non_attacking_pairs(state):
    
    total_pairs = 28  # Number of pairs of queens in  8-queens 
    attacking_pairs = 0
    for i in range(len(state)):
        for j in range(i + 1, len(state)):
            if state[i] == state[j] or abs(state[i] - state[j]) == abs(i - j):
                attacking_pairs += 1
    return total_pairs - attacking_pairs

def hill_climbing(state):
    """Perform hill-climbing to maximize the number of non-attacking pairs."""
    current_non_attacking_pairs = calculate_non_attacking_pairs(state)
    steps = 0
    
    while True:
        steps += 1
        neighbors = []
        for col in range(8):
            for row in range(8):
                if state[col] != row:
                    new_state = list(state)
                    new_state[col] = row
                    neighbors.append((new_state, calculate_non_attacking_pairs(new_state)))
        
        # Sort neighbors based on number of non-attacking pairs
        neighbors.sort(key=lambda x: x[1], reverse=True)
        
        # Find the best neighbor
        best_neighbor, best_non_attacking_pairs = neighbors[0]
        
        # Check for improvement
        if best_non_attacking_pairs > current_non_attacking_pairs:
            state = best_neighbor
            current_non_attacking_pairs = best_non_attacking_pairs
        else:
            # No improvement, return current state
            return state, current_non_attacking_pairs, steps

def run_simulations(num_simulations):
    """Run multiple simulations and return the best results."""
    successes = 0
    steps_successful = []
    steps_stuck = []
    best_solution = None
    best_non_attacking_pairs = 0
    best_steps = float('inf')
    best_probability = 0
    
    for _ in range(num_simulations):
        state = random_queen_state()
        final_state, final_non_attacking_pairs, steps = hill_climbing(state)
        
        if final_non_attacking_pairs == 28:
            successes += 1
            steps_successful.append(steps)
        else:
            steps_stuck.append(steps)
        
        # Track the best solution found
        if (final_non_attacking_pairs > best_non_attacking_pairs or
            (final_non_attacking_pairs == best_non_attacking_pairs and steps < best_steps)):
            best_solution = final_state
            best_non_attacking_pairs = final_non_attacking_pairs
            best_steps = steps
    
    success_probability = successes / num_simulations
    avg_steps_success = sum(steps_successful) / successes if successes > 0 else 0
    avg_steps_stuck = sum(steps_stuck) / (num_simulations - successes) if (num_simulations - successes) > 0 else 0
    
    return (success_probability, avg_steps_success, avg_steps_stuck, 
            best_solution, best_non_attacking_pairs, best_steps)

def display_board(state):
    """Display the board with queens and dots."""
    board = [['.' for _ in range(8)] for _ in range(8)]
    for col, row in enumerate(state):
        board[row][col] = 'Q'
    
    for row in board:
        print(' '.join(row))

# Parameters
num_simulations = 1000
best_results = None
best_probability = 0

# Run simulations
print("Running hill-climbing simulations...")
results = run_simulations(num_simulations)

# Check for better results
if results[0] > best_probability:
    best_probability = results[0]
    best_results = results


if best_results:
    print("\nBest Results Found:")
    print(f"Success Probability: {best_results[0] * 100:.2f}%")
    print(f"Average Steps when Successful: {best_results[1]}")
    print(f"Average Steps when Stuck: {best_results[2]}")
    print("\nBest Solution Found:")
    display_board(best_results[3])
    print(f"Non-Attacking Pairs: {best_results[4]}")
    print(f"Steps to Find Solution: {best_results[5]}")


Running hill-climbing simulations...

Best Results Found:
Success Probability: 16.70%
Average Steps when Successful: 5.1017964071856285
Average Steps when Stuck: 4.050420168067227

Best Solution Found:
. . . . . . Q .
. . . Q . . . .
. Q . . . . . .
. . . . Q . . .
. . . . . . . Q
Q . . . . . . .
. . Q . . . . .
. . . . . Q . .
Non-Attacking Pairs: 28
Steps to Find Solution: 3


In [2]:
#hill-climbing with sideways moves
import random
import pandas as pd

def random_queen_state():
    
    return [random.randint(0, 7) for _ in range(8)]

def calculate_non_attacking_pairs(state):
    
    total_pairs = 28  # Number of pairs of queens in an 8-queens 
    attacking_pairs = 0
    for i in range(len(state)):
        for j in range(i + 1, len(state)):
            if state[i] == state[j] or abs(state[i] - state[j]) == abs(i - j):
                attacking_pairs += 1
    return total_pairs - attacking_pairs

def hill_climbing_with_sideways(state, max_sideways_moves):
    
    current_non_attacking_pairs = calculate_non_attacking_pairs(state)
    steps = 0
    sideways_moves = 0
    
    while True:
        steps += 1
        neighbors = []
        for col in range(8):
            for row in range(8):
                if state[col] != row:
                    new_state = list(state)
                    new_state[col] = row
                    neighbors.append((new_state, calculate_non_attacking_pairs(new_state)))
        
        # Sort neighbors based on number of non-attacking pairs
        neighbors.sort(key=lambda x: x[1], reverse=True)
        
        # Find the best neighbor
        best_neighbor, best_non_attacking_pairs = neighbors[0]
        
        # Check for improvement
        if best_non_attacking_pairs > current_non_attacking_pairs:
            state = best_neighbor
            current_non_attacking_pairs = best_non_attacking_pairs
            sideways_moves = 0  # Reset sideways moves
        else:
            if sideways_moves < max_sideways_moves:
                sideways_moves += 1
                # Try a sideways move by choosing randomly among neighbors with same score
                sideways_neighbors = [n for n in neighbors if n[1] == current_non_attacking_pairs]
                if sideways_neighbors:
                    state = random.choice(sideways_neighbors)[0]
                else:
                    # No sideways moves possible, return current state
                    return state, current_non_attacking_pairs, steps, sideways_moves
            else:
                # No improvement and limit reached, return current state
                return state, current_non_attacking_pairs, steps, sideways_moves

def run_simulations_with_sideways(num_simulations, max_sideways_moves):
    """Run multiple simulations with sideways moves and return the results."""
    successes = 0
    steps_successful = []
    steps_stuck = []
    sideways_moves_successful = []
    sideways_moves_stuck = []
    
    for _ in range(num_simulations):
        state = random_queen_state()
        final_state, final_non_attacking_pairs, steps, sideways_moves = hill_climbing_with_sideways(state, max_sideways_moves)
        
        if final_non_attacking_pairs == 28:
            successes += 1
            steps_successful.append(steps)
            sideways_moves_successful.append(sideways_moves)
        else:
            steps_stuck.append(steps)
            sideways_moves_stuck.append(sideways_moves)
    
    success_probability = successes / num_simulations
    avg_steps_success = sum(steps_successful) / successes if successes > 0 else 0
    avg_steps_stuck = sum(steps_stuck) / (num_simulations - successes) if (num_simulations - successes) > 0 else 0
    avg_sideways_moves_success = sum(sideways_moves_successful) / successes if successes > 0 else 0
    avg_sideways_moves_stuck = sum(sideways_moves_stuck) / (num_simulations - successes) if (num_simulations - successes) > 0 else 0
    
    success_percentage = success_probability * 100
    stuck_probability = (1 - success_probability) * 100
    
    return (success_percentage, stuck_probability, avg_steps_success, 
            avg_steps_stuck, avg_sideways_moves_success, avg_sideways_moves_stuck)

def display_results(results):
    """Display results in a tabular format."""
    df = pd.DataFrame(results, columns=[
        'Max Sideways Moves', 'Success (%)', 'Stuck Probability (%)', 
        'Avg Steps (Successful)', 'Avg Steps (Stuck)', 
        'Avg Sideways Moves (Successful)', 'Avg Sideways Moves (Stuck)'
    ])
    print(df.to_string(index=False))

# Parameters
num_simulations = 1000
sideways_limits = [10, 50, 100, 200, 350, 500]
results = []

# Run simulations for different sideways move limits
for limit in sideways_limits:
    print(f"\nRunning simulations with {limit} sideways moves...")
    result = run_simulations_with_sideways(num_simulations, limit)
    results.append([limit] + list(result))


display_results(results)



Running simulations with 10 sideways moves...

Running simulations with 50 sideways moves...

Running simulations with 100 sideways moves...

Running simulations with 200 sideways moves...

Running simulations with 350 sideways moves...

Running simulations with 500 sideways moves...
 Max Sideways Moves  Success (%)  Stuck Probability (%)  Avg Steps (Successful)  Avg Steps (Stuck)  Avg Sideways Moves (Successful)  Avg Sideways Moves (Stuck)
                 10         58.7                   41.3                8.628620          14.876513                              1.0                    9.542373
                 50         88.9                   11.1               15.995501          44.846847                              1.0                   39.405405
                100         93.5                    6.5               20.549733          64.000000                              1.0                   58.876923
                200         95.4                    4.6               22.1

In [37]:
import random
import pandas as pd
import time

def calculate_non_attacking_pairs(state):
    
    non_attacking_pairs = 28  
    attacking_pairs = 0
    
    for i in range(8):
        for j in range(i + 1, 8):
            # Check if queens are in the same row or on the same diagonal
            if state[i] == state[j] or abs(state[i] - state[j]) == abs(i - j):
                attacking_pairs += 1

    return non_attacking_pairs - attacking_pairs

def stochastic_hill_climbing(state, max_steps_without_improvement):
    # Stochastic Hill Climbing with random neighbor selection
    current_state = state
    current_non_attacking_pairs = calculate_non_attacking_pairs(current_state)
    steps = 0
    steps_without_improvement = 0
    solutions = set()
    
    while steps_without_improvement < max_steps_without_improvement:
        steps += 1
        neighbors = []
        
        # Generate neighbors by modifying one queen's position at a time
        for col in range(8):
            for row in range(8):
                if current_state[col] != row:
                    new_state = list(current_state)
                    new_state[col] = row
                    neighbors.append((tuple(new_state), calculate_non_attacking_pairs(new_state)))
        
        if not neighbors:
            break
        
        # Randomly select a neighbor
        current_state, current_non_attacking_pairs = random.choice(neighbors)
        
        if current_non_attacking_pairs == 28:
            solutions.add(tuple(current_state))
            steps_without_improvement = 0  # Reset when a solution is found
        else:
            steps_without_improvement += 1
        
    return solutions, steps

def random_queen_state():
    
    return tuple(random.randint(0, 7) for _ in range(8))

def run_stochastic_simulation(num_simulations, max_steps_without_improvement):
    
    all_solutions = set()
    total_steps = 0
    success_count = 0
    failure_steps = []
    success_steps = []
    total_time = 0
    
    for _ in range(num_simulations):
        start_time = time.time()
        
        state = random_queen_state()
        solutions, steps = stochastic_hill_climbing(state, max_steps_without_improvement)
        
        end_time = time.time()
        elapsed_time = end_time - start_time
        total_time += elapsed_time
        
        if solutions:
            success_count += 1
            all_solutions.update(solutions)
            success_steps.append(steps)
        else:
            failure_steps.append(steps)
        
        total_steps += steps
    
    success_percentage = (success_count / num_simulations) * 100
    stuck_probability = ((num_simulations - success_count) / num_simulations) * 100
    avg_steps = total_steps / num_simulations
    avg_success_steps = sum(success_steps) / len(success_steps) if success_steps else 0
    avg_failure_steps = sum(failure_steps) / len(failure_steps) if failure_steps else 0
    avg_time = total_time / num_simulations
    
    return success_percentage, stuck_probability, len(all_solutions), avg_steps, avg_success_steps, avg_failure_steps, avg_time

def display_stochastic_results(results):
    """Display Stochastic Hill Climbing results in a tabular format."""
    df = pd.DataFrame(
        {
            'Algorithm': ['Stochastic Hill-Climbing'],
            'Success (%)': [results[0]],
            'Stuck Probability (%)': [results[1]],
            'Unique Solutions Found': [results[2]],
            'Avg Steps per Simulation': [results[3]],
            'Avg Steps on Success': [results[4]],
            'Avg Steps on Failure': [results[5]],
            'Avg Time per Simulation (s)': [results[6]]
        }
    )
    
    print(df.to_string(index=False))

# Parameters
num_simulations = 1000  # can Reduce the number of simulations for faster results but with less success rate
max_steps_without_improvement = 500  # Adjusted based on performance

# Run Stochastic Hill Climbing simulation
stochastic_results = run_stochastic_simulation(num_simulations, max_steps_without_improvement)
display_stochastic_results(stochastic_results)


               Algorithm  Success (%)  Stuck Probability (%)  Unique Solutions Found  Avg Steps per Simulation  Avg Steps on Success  Avg Steps on Failure  Avg Time per Simulation (s)
Stochastic Hill-Climbing          0.4                   99.6                       4                   501.183                795.75                 500.0                     0.152896


In [33]:
import random
import pandas as pd

def calculate_non_attacking_pairs(state):
    
    non_attacking_pairs = 28  # Max possible pairs for 8 queens
    attacking_pairs = 0
    
    for i in range(8):
        for j in range(i + 1, 8):
            if state[i] == state[j] or abs(state[i] - state[j]) == abs(i - j):
                attacking_pairs += 1

    return non_attacking_pairs - attacking_pairs

def random_queen_state():
    
    return [random.randint(0, 7) for _ in range(8)]

def random_neighbor(state): #Generate a random neighbor state by changing the position of one queen
    
    col = random.randint(0, 7)  
    row = random.randint(0, 7)  
    while state[col] == row:
        row = random.randint(0, 7)  
    
    new_state = list(state)
    new_state[col] = row
    return new_state

def random_walk_hill_climbing(state, max_steps_without_improvement, sideways_moves_limit): 
    #Random-Walk Hill Climbing with sideways moves
    current_state = state
    current_non_attacking_pairs = calculate_non_attacking_pairs(current_state)
    steps = 0
    sideways_moves = 0
    
    while steps < max_steps_without_improvement:
        steps += 1
        
        
        neighbor = random_neighbor(current_state)
        neighbor_non_attacking_pairs = calculate_non_attacking_pairs(neighbor)
        
        # Accept the neighbor even if it's not an improvement (random walk)
        current_state = neighbor
        current_non_attacking_pairs = neighbor_non_attacking_pairs
        
        if current_non_attacking_pairs == 28:  #  solution found case 
            return current_state, current_non_attacking_pairs, steps, sideways_moves
        
        sideways_moves += 1
        if sideways_moves >= sideways_moves_limit:
            break

    # Return the state even if it's not optimal (stuck case)
    return current_state, current_non_attacking_pairs, steps, sideways_moves

def run_random_walk_simulation(num_simulations, max_steps_without_improvement, sideways_moves_limit):
    
    success_count = 0
    total_steps_success = 0
    total_steps_stuck = 0
    total_sideways_success = 0
    total_sideways_stuck = 0
    stuck_count = 0
    
    for _ in range(num_simulations):
        final_state, final_non_attacking_pairs, steps, sideways_moves = random_walk_hill_climbing(
            random_queen_state(), max_steps_without_improvement, sideways_moves_limit
        )
        
        if final_non_attacking_pairs == 28:  # Success
            success_count += 1
            total_steps_success += steps
            total_sideways_success += sideways_moves
        else:  # Stuck
            stuck_count += 1
            total_steps_stuck += steps
            total_sideways_stuck += sideways_moves

    # Calculate success rate, average steps, and sideways moves
    success_rate = (success_count / num_simulations) * 100
    stuck_rate = (stuck_count / num_simulations) * 100

    avg_steps_success = total_steps_success / success_count if success_count > 0 else 0
    avg_steps_stuck = total_steps_stuck / stuck_count if stuck_count > 0 else 0

    avg_sideways_success = total_sideways_success / success_count if success_count > 0 else 0
    avg_sideways_stuck = total_sideways_stuck / stuck_count if stuck_count > 0 else 0
    
    return [
        success_rate, stuck_rate, 
        avg_steps_success, avg_steps_stuck, 
        avg_sideways_success, avg_sideways_stuck
    ]

# Parameters
num_simulations = 7500
max_steps_without_improvement = 5000
sideways_moves_limits = [1000, 2500, 5000]

# Run Random-Walk Hill Climbing simulation for each limit
results = []
for limit in sideways_moves_limits:
    rw_results = run_random_walk_simulation(num_simulations, max_steps_without_improvement, limit)
    results.append([limit] + rw_results)

# Create a DataFrame to display the results
rw_df = pd.DataFrame(
    results,
    columns=[
        'Sideways Moves Limit', 'Success (%)', 'Stuck Probability (%)', 
        'Avg Steps (Successful)', 'Avg Steps (Stuck)', 
        'Avg Sideways Moves (Successful)', 'Avg Sideways Moves (Stuck)'
    ]
)

# Print the results
print(rw_df.to_string(index=False))


 Sideways Moves Limit  Success (%)  Stuck Probability (%)  Avg Steps (Successful)  Avg Steps (Stuck)  Avg Sideways Moves (Successful)  Avg Sideways Moves (Stuck)
                 1000     0.533333              99.466667               548.70000             1000.0                        547.70000                      1000.0
                 2500     1.346667              98.653333              1224.49505             2500.0                       1223.49505                      2500.0
                 5000     2.480000              97.520000              2442.88172             5000.0                       2441.88172                      5000.0
